In [3]:
import pandas as pd
import numpy as np
import plotly
from sqlalchemy import create_engine

In [5]:
engine = create_engine('sqlite:///%s' % '../data/coviddb')
df = pd.read_sql_table('CovidData', engine)

In [17]:
# df.
df.describe()

,county_fips,PopulationCount,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CHD,...,MAMMOUSE,MHLTH,OBESITY,PAPTEST,PHLTH,SLEEP,STROKE,TEETHLOST,case1m,death1m
count,285.000000,2.850000e+02,285.000000,285.000000,285.000000,285.000000,285.000000,285.000000,285.000000,285.000000,...,285.000000,285.000000,285.000000,262.000000,285.000000,285.000000,285.000000,285.000000,285.000000,285.000000
mean,28932.898246,6.633935e+05,0.158021,0.239002,0.173742,0.311911,0.668271,0.058342,0.096665,0.060631,...,0.761728,0.132476,0.317517,0.804137,0.128581,0.358395,0.032583,0.160527,6413.345856,361.437427
std,16538.990608,1.166812e+06,0.055967,0.035635,0.027307,0.047545,0.042364,0.005412,0.012097,0.009704,...,0.033880,0.018207,0.053935,0.025603,0.022782,0.046007,0.007244,0.045893,9767.250806,688.586802
min,1073.000000,8.483400e+04,0.064708,0.152227,0.083759,0.215647,0.553500,0.042500,0.074436,0.042000,...,0.637000,0.085000,0.168069,0.723656,0.076015,0.250781,0.018269,0.066000,217.001814,0.000000
25%,13095.000000,1.827020e+05,0.118500,0.210500,0.155500,0.277000,0.638500,0.054936,0.087500,0.053500,...,0.740000,0.119597,0.278500,0.789000,0.112000,0.327000,0.027000,0.128000,1436.790338,37.287925
50%,29019.000000,3.222480e+05,0.147500,0.240500,0.174000,0.310000,0.673500,0.058500,0.095000,0.061000,...,0.764500,0.133000,0.321000,0.806449,0.129500,0.355500,0.032000,0.159500,3016.034475,91.718885
75%,45091.000000,6.121740e+05,0.182672,0.262000,0.190792,0.341500,0.697000,0.062000,0.104000,0.066917,...,0.785000,0.144000,0.355542,0.823495,0.143000,0.387191,0.036116,0.187112,6339.928287,312.683058
max,56021.000000,1.401483e+07,0.415000,0.346000,0.263500,0.449000,0.763000,0.075500,0.136000,0.092500,...,0.827500,0.183758,0.452000,0.857000,0.199000,0.497000,0.057500,0.316000,63447.735038,4557.228165


In [22]:
df.corr()

,county_fips,PopulationCount,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CHD,...,MAMMOUSE,MHLTH,OBESITY,PAPTEST,PHLTH,SLEEP,STROKE,TEETHLOST,case1m,death1m
county_fips,1.000000,-0.149697,0.112161,0.143130,0.010040,0.040107,0.161467,-0.024377,0.098530,0.014516,...,-0.066915,0.032996,0.223546,-0.051767,-0.009773,0.048259,-0.012121,0.031645,-0.033363,-0.053356
PopulationCount,-0.149697,1.000000,0.063447,-0.274319,0.021868,-0.107186,-0.171512,-0.227107,-0.124517,-0.162537,...,0.097154,-0.134934,-0.200044,0.055564,-0.072823,0.019311,-0.086145,-0.138326,-0.078326,-0.073946
ACCESS2,0.112161,0.063447,1.000000,0.003982,-0.444630,0.373501,0.198687,-0.420439,-0.114235,0.386513,...,-0.132253,0.335696,0.471918,-0.366341,0.521307,0.317294,0.423334,0.422384,0.049209,0.015260
ARTHRITIS,0.143130,-0.274319,0.003982,1.000000,-0.297667,0.711258,0.645637,0.493382,0.682950,0.799673,...,0.002165,0.683639,0.665666,-0.071493,0.684383,0.539811,0.725658,0.672309,0.082168,0.147939
BINGE,0.010040,0.021868,-0.444630,-0.297667,1.000000,-0.523409,-0.246949,0.140548,-0.228549,-0.440903,...,0.052310,-0.470113,-0.384729,0.272593,-0.530712,-0.434092,-0.500440,-0.491046,-0.077499,-0.105594
BPHIGH,0.040107,-0.107186,0.373501,0.711258,-0.523409,1.000000,0.788724,0.049923,0.480071,0.768554,...,0.234375,0.665030,0.779442,0.040759,0.722864,0.771117,0.869530,0.783885,0.120828,0.159657
BPMED,0.161467,-0.171512,0.198687,0.645637,-0.246949,0.788724,1.000000,0.174571,0.297592,0.604743,...,0.380346,0.406613,0.625567,0.142815,0.442832,0.564754,0.645294,0.562216,0.117308,0.143675
CANCER,-0.024377,-0.227107,-0.420439,0.493382,0.140548,0.049923,0.174571,1.000000,0.110269,0.288084,...,-0.095468,-0.006810,-0.046866,0.048784,-0.004857,-0.178661,0.049609,-0.064017,-0.162442,-0.122677
CASTHMA,0.098530,-0.124517,-0.114235,0.682950,-0.228549,0.480071,0.297592,0.110269,1.000000,0.560279,...,0.039653,0.717982,0.488868,-0.119207,0.609712,0.555439,0.590739,0.624484,0.233260,0.303030
CHD,0.014516,-0.162537,0.386513,0.799673,-0.440903,0.768554,0.604743,0.288084,0.560279,1.000000,...,-0.058307,0.807831,0.743357,-0.260285,0.900548,0.630541,0.880458,0.843344,0.154332,0.196401


In [20]:
import plotly.express as px
fig = px.scatter(df, x="DENTAL", y="death1m", color="state_id")
fig.show()

In [18]:
lookup = pd.read_sql_table('MeasureLookup', engine)


In [19]:
lookup

,MeasureId,Measure,Short_Question_Text
0,BPHIGH,High blood pressure among adults aged >=18 Years,High Blood Pressure
1,CHOLSCREEN,Cholesterol screening among adults aged >=18 Y...,Cholesterol Screening
2,HIGHCHOL,High cholesterol among adults aged >=18 Years ...,High Cholesterol
3,BPMED,Taking medicine for high blood pressure contro...,Taking BP Medication
4,TEETHLOST,All teeth lost among adults aged >=65 Years,Teeth Loss
5,STROKE,Stroke among adults aged >=18 Years,Stroke
6,OBESITY,Obesity among adults aged >=18 Years,Obesity
7,ARTHRITIS,Arthritis among adults aged >=18 Years,Arthritis
8,CASTHMA,Current asthma among adults aged >=18 Years,Current Asthma
9,DIABETES,Diagnosed diabetes among adults aged >=18 Years,Diabetes
